In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
import docker
from docker.errors import NotFound
from docker.models.containers import Container
from docker.types import Mount

GEM_ALADDIN_WORKSPACE = "gem5-aladdin-workspace"

CONTAINER_NAME = "gem5aladdin"
relative_path = "gem5/ase/10/botorch/20201010"
task = "aes_aes"

workdir = f"/workspace/{relative_path}/{task}/0/"

In [5]:
def _get_or_create_docker_container(client) -> Container:
    try:
        container = client.containers.get(CONTAINER_NAME)
    except NotFound as e:
        cmd = None
        # Check if we need to create the volume and compile all libraries
        persistant_volume = Mount(source=GEM_ALADDIN_WORKSPACE, target="/workspace")
        container = client.containers.run(
            image="xyzsam/gem5-aladdin",
            command=cmd,
            name=CONTAINER_NAME,
            detach=True,
            mounts=[persistant_volume],
            tty=True,
            remove=True,
        )
    return container

client = docker.from_env()
# First check if volume exist first if not, create it and run build commands
gem5_container = _get_or_create_docker_container(client)

In [ ]:
gem5_container.exec_run(workdir="/workspace/", cmd=f"mkdir -p '{relative_path}'")

In [69]:
gem5_container.exec_run(workdir = "/workspace/gem5-aladdin/sweeps/benchmarks",
                        cmd=["/workspace/gem5-aladdin/sweeps/generate_design_sweeps.py",
                             "/workspace/gem5-aladdin/sweeps/benchmarks/machsuite_test.xe"])

ExecResult(exit_code=0, output=b'use benchmarks.designsweeptypes.Gem5DesignSweep\nbegin Gem5DesignSweep single\nuse benchmarks.machsuite.fft_transpose\ngenerate configs\ngenerate dma_trace\ngenerate gem5_binary\nset output_dir "/workspace/gem5/ase/10/botorch/20201010"\nset source_dir "/workspace/gem5-aladdin/src/aladdin/MachSuite"\nset simulator "gem5-cpu"\nset memory_type "spad"\nset cycle_time 30\nset tlb_page_size 4096\nset cache_line_sz 64\nset memory_type for fft_transpose.twiddles8.reversed8 "spad"\nset memory_type for fft_transpose.data_x "spad"\nset memory_type for fft_transpose.data_y "spad"\nset partition_type for fft_transpose.twiddles8.reversed8 "complete"\nset partition_type for fft_transpose.data_x "complete"\nset partition_type for fft_transpose.data_y "complete"\nset memory_type for fft_transpose.work_x "spad"\nset memory_type for fft_transpose.work_y "spad"\nset memory_type for fft_transpose.DATA_x "spad"\nset memory_type for fft_transpose.DATA_y "spad"\nset memory_typ

In [6]:
with open("resources/machsuite_tasks_constants.xe") as constants_file:
    constants_vals = constants_file.read()

FileNotFoundError: [Errno 2] No such file or directory: 'resources/machsuite_tasks_constants.xe'

In [7]:
import re

evaluation_configs = {
    "cycle_time": 50,
    "cache_line_sz": 32
}
evaluation_configs = [
    f"set {inp} {val}" for inp, val in evaluation_configs.items()
]

task_pattern = re.compile(f"set \w* for \S*{task}\S* .*")
task_constants = task_pattern.findall(constants_vals)

In [67]:
from pathlib import Path
from string import Template

with open(Path("resources") / "machsuite_params_template.xe") as bench_template_f:
    benchmark_template = Template(bench_template_f.read())
benchmark_template_filled = benchmark_template.substitute(
    {
        "output_dir": "test_out",
        "source_dir": str(Path("hello")),
        "simulator": "yes",
        "memory_type": "no",
        "generation_commands": "command",
        "task": str(task),
        "evaluation_configs": "\n".join(evaluation_configs),
        "task_constants": "\n".join(task_constants)
    }
)

In [68]:
with open("test_out.xe", "w") as test_out:
    test_out.write(benchmark_template_filled)

In [92]:


gem5_container.exec_run(workdir = f"/workspace/{relative_path}", cmd = [
    "bash", "-c", f"echo '{benchmark_template_filled}' > test.xe"])

ExecResult(exit_code=0, output=b'')

In [72]:

print(f"/workspace/{relative_path}")

/workspace/gem5/ase/10/botorch/20201010


In [8]:

gem5_container.exec_run(workdir = f"/workspace/{relative_path}/{task}/0/", cmd = ["bash", "run.sh"])

ExecResult(exit_code=0, output=b'')

In [14]:
gem5_container.exec_run(workdir =workdir, cmd = "rm -rf outputs/stats.db outputs/ffs")

ReadTimeout: UnixHTTPConnectionPool(host='localhost', port=None): Read timed out. (read timeout=60)

In [ ]:
bits, stats = gem5_container.get_archive(path = f"{workdir}/outputs")

In [20]:
for chunk in bits:
    print(chunk)

In [16]:
out_dir  = f"{workdir}/outputs"

In [34]:
test = gem5_container.exec_run(workdir = out_dir, cmd = ["bash", "-c" , "cat *_summary"])
test.output

b'===============================\n        Aladdin Results        \n===============================\nRunning : /workspace/gem5/ase/10/botorch/20201010/fft_transpose/0/outputs/fft_transpose\nTop level function: fft1D_512\nCycle : 24156 cycles\nUpsampled Cycle : 0 cycles\nAvg Power: 22.0119 mW\nIdle FU Cycles: 4877 cycles\nAvg FU Power: 20.9856 mW\nAvg FU Dynamic Power: 12.6527 mW\nAvg FU leakage Power: 8.33294 mW\nAvg MEM Power: 1.02628 mW\nAvg MEM Dynamic Power: 0.0888076 mW\nAvg MEM Leakage Power: 0.937476 mW\nTotal Area: 962188 uM^2\nFU Area: 907193 uM^2\nMEM Area: 54995.5 uM^2\nNum of Double Precision FP Multipliers: 33\nNum of Double Precision FP Adders: 27\nNum of Trigonometric Units: 14\nNum of Bit-wise Operators (32-bit): 9\nNum of Shifters (32-bit): 6\nNum of Registers (32-bit): 544\n===============================\n        Aladdin Results        \n===============================\n'

In [24]:
with open("test_summary", 'wb') as outf:
    outf.write(test.output)


files_to_capture = [
    "stderr",
    "stdout",
    "stats.txt"
    "*_cache_stats.txt"
    "*_spad_stats.txt"
    "*_summary"
]

In [52]:
cmd_exe = "&&".join(["cd gem5-aladdin", "git pull origin master", "cd /workspace", "touch hello-again"])

gem5_container.exec_run(workdir = "/workspace/", cmd = ["bash",
                                                   "-c" ,
                                                   cmd_exe])

ExecResult(exit_code=0, output=b'From https://github.com/harvard-acc/gem5-aladdin\n * branch                master     -> FETCH_HEAD\nAlready up to date.\n')

In [54]:
cmd_exe

'cd gem5-aladdin&&git pull origin master&&cd /workspace&&touch hello-again'